In [ ]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.3.0 pyspark-shell'


import sys
import time
import json
from pymongo import MongoClient
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

def sendDataToDB(iter):
    climate=[]
    tera=[]
    aqua=[]
    
    for i in iter:
        val=i[1]
        id=val.split(",")[0]
        if(id=="1"):
            climate.append(i)
        elif(id.strip()=="2"):
            aqua.append(i)
        elif(id.strip()=="3"):
            tera.append(i)
    client = MongoClient()
    db = client.climate_streaming
    assignment = db.assignment


    for cl in climate:
        climate_hash=cl[1].split(",")[9]
        
        if(len(climate) >=1 and len(aqua) == 0 and len(tera) == 0):
            entry = {
                'time' : cl[1].split(",")[1],
            'latitude' :float(cl[1].split(",")[2]),
            'longitude':float(cl[1].split(",")[3]),
            'air_temperature_celcious':float(cl[1].split(",")[4]),
            'relative_humidity': float(cl[1].split(",")[5]),
            'windspeed_knots' :float(cl[1].split(",")[6]),
            'max_wind_speed' : float(cl[1].split(",")[7]),
            'precipitation' : cl[1].split(",")[8],
            'climate_hotspot' : []
        }
            print("case 1:",entry)
            assignment.insert_one(entry)
            
        elif(len(climate) >=1 and len(aqua) >= 1 and len(tera) == 0):
            for aq in aqua:
                aq_hash= aq[1].split(",")[6]
                if(aq_hash == climate_hash):
                    aqua_entry = {
                    'confidence' : float(aq[1].split(",")[4]),
                    'surface_temperature_celcious' : float(aq[1].split(",")[5])
                }
                    
                    entry = {
                        'time' : cl[1].split(",")[1],
            'latitude' :float(cl[1].split(",")[2]),
            'longitude':float(cl[1].split(",")[3]),
            'air_temperature_celcious':float(cl[1].split(",")[4]),
            'relative_humidity': float(cl[1].split(",")[5]),
            'windspeed_knots' :float(cl[1].split(",")[6]),
            'max_wind_speed' : float(cl[1].split(",")[7]),
            'precipitation' : cl[1].split(",")[8],
            'climate_hotspot' : aqua_entry 
                        
        }
                    print("case 2:",entry)
                    assignment.insert_one(entry)
        elif(len(climate) >=1 and len(aqua) == 0 and len(tera) >= 1):
            for tr in tera:
                tr_hash= tr[1].split(",")[6]
                if(tr_hash == climate_hash):
                    tera_entry = {
                    'confidence' : float(tr[1].split(",")[4]),
                    'surface_temperature_celcious' : float(tr[1].split(",")[5])
                }

                    entry = {
                        'time' : cl[1].split(",")[1],
            'latitude' :float(cl[1].split(",")[2]),
            'longitude':float(cl[1].split(",")[3]),
            'air_temperature_celcious':float(cl[1].split(",")[4]),
            'relative_humidity': float(cl[1].split(",")[5]),
            'windspeed_knots' :float(cl[1].split(",")[6]),
            'max_wind_speed' : float(cl[1].split(",")[7]),
            'precipitation' : cl[1].split(",")[8],
            'climate_hotspot' : tera_entry 

        }
                    print("case 3:",entry)
                    assignment.insert_one(entry)
                    
            
    
        elif(len(climate) >=1 and len(tera)>=1 and len(aqua)>=1):
            for tr in tera:
                tera_hash= tr[1].split(",")[6]
                for aq in aqua:
                    terra_list = []
                    aq_hash= aq[1].split(",")[6]
                    if(tera_hash==aq_hash and climate_hash == tera_hash):
                        aqua_temp = int(aq[1].split(",")[4].strip())
                        tr_temp = int(tr[1].split(",")[4].strip())
                        aqua_conf = int(aq[1].split(",")[5].strip())
                        tr_conf = int(tr[1].split(",")[5].strip())
                        
                        avg_temp = (aqua_temp + tr_temp) / 2
                        avg_conf = (aqua_conf + tr_conf) / 2 
                        
                        tera_entry = {
                'confidence' : avg_conf,
                'surface_temperature_celcious' : avg_temp
            }
                        
                        terra_list.append(tera_entry)
                        

                        entry = {
                            'time' : cl[1].split(",")[1],
            'latitude' :float(cl[1].split(",")[2]),
            'longitude':float(cl[1].split(",")[3]),
            'air_temperature_celcious':float(cl[1].split(",")[4]),
            'relative_humidity': float(cl[1].split(",")[5]),
            'windspeed_knots' :float(cl[1].split(",")[6]),
            'max_wind_speed' : float(cl[1].split(",")[7]),
            'precipitation' : cl[1].split(",")[8],
            'climate_hotspot' : terra_list

            }
                        print("case 4:",entry)
                        assignment.insert_one(entry)
    client.close()
                        

n_secs = 10
topic = "climate"

conf = SparkConf().setAppName("KafkaStreamProcessor").setMaster("local[2]")
sc = SparkContext.getOrCreate()
if sc is None:
    sc = SparkContext(conf=conf)
sc.setLogLevel("WARN")
ssc = StreamingContext(sc, n_secs)
    
kafkaStream = KafkaUtils.createDirectStream(ssc, [topic], {
                        'bootstrap.servers':'127.0.0.1:9092', 
                        'group.id':'week11-group', 
                        'fetch.message.max.bytes':'15728640',
                        'auto.offset.reset':'largest'})

  
lines = kafkaStream.foreachRDD(lambda rdd: rdd.foreachPartition(sendDataToDB))


ssc.start()
time.sleep(600) # Run stream for 10 minutes just in case no detection of producer
# ssc.awaitTermination()
ssc.stop(stopSparkContext=True,stopGraceFully=True)